In [1]:
import zipfile
import tarfile
import json
import os
import gzip as gz
import pandas as pd
import numpy as np
import folium
from pathlib import Path
from shapely.geometry import *
from pandas import DataFrame 
from folium import plugins

from readData import ReadData
from createDataframe import CreateDF

#jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10

## Var definition

In [2]:
#2016
#2017-2018

year = '2016'
directory_base = '../'
tar_directory = '../tar_files'
coords = box(43.7359, 10.4269, 43.6955, 10.3686)
lat = [43.7359, 43.6955]
lon = [10.4269, 10.3686]
lat_mean = np.mean(lat)
lon_mean = np.mean(lon)

## Download and read files

Salvo i dati in un file json con un dizionario per ogni riga dove il dizionario descrive il giorno (come nell'input), mantenendo solo i campi di interesse, ovvero:
- text 
- id 
- retweeted 
- coordinates = represents the geographic location of this Tweet as reported by the user or client application
- timestamp_ms
- entities 
- retweet_count  
- in_reply_to_user_id 
- user 
- geo = is it deprecated? 
- in_reply_to_user_id_str
- lang 
- created_at 
- in_reply_to_status_id_str 
- place = when present, indicates that the tweet is associated (but not necessarily originating from) a Place

#### Campi non considerati:

<li>contributors</li>
<li>truncated</li>
<li>is_quote_status</li>
<li>source</li>
<li>in_reply_to_screen_name</li>
<li>id_str</li>
<li>favorited</li>
<li>filter_level</li>

Fonte: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet

In [3]:
#read data
for tar_file in os.listdir(tar_directory):
    if tar_file.endswith('.tar.gz') and year in tar_file:
        tar=tarfile.open(tar_directory+'/'+tar_file)
        #salvo i file della cartella in tarfiles
        tarfiles = tar.getmembers()
        #creo anche un pathName per avere la directory e salvare i file
        pathName = "../json/dataPerID_"+year+".json"
        UID='uidFiles_'+year
        emptyTweetDictionary = {}
        # chiamo la classe ReadData
        rd = ReadData(tar, tarfiles, pathName, UID, emptyTweetDictionary)
        #creo il dizionario di quell'anno contenente tutti i tweet formato {ID tweet : tweet}
        tweetDictionary = rd.dataPerID(tar, tarfiles, pathName, emptyTweetDictionary)
        rd.createUIDFiles(UID, tweetDictionary)

Number of tweets:  12843
Number of files:  14


## Filter by coordinates and Creation of the dataframe

Filtro i dati in formato json ottenuti nella prima fase, in base alle loro coordinate: se si trovano all'interno di un poligono predefinito (l'area che ci interessa), allora li filtro, li pulisco e prendo solo le informazioni a noi utili (nome utente, coordinate, ecc.) e li inserisco in un dataframe, che chiamo "df".

In [4]:
#filter data to create the df
for folder in os.listdir(directory_base): #scorro le cartelle nella directory di base
    folder_name = os.path.basename(folder) #prendo il nome della cartella
    if folder_name.startswith('uidFiles') and year in folder_name: #se inizia per uidFiles, la analizzo
        #creo il dataframe con le colonne
        df = pd.DataFrame(columns=['Screen_name','UserID', 'TweetID', 'Coords', 'Lat', 'Lon', 'Created_At', 'Text'])
        #salvo i file su entries dalla cartella "uidFiles"
        entries = os.listdir('../'+folder_name)
        print('In '+folder_name+' ci sono '+str(len(entries))+' files') #stampo quanti file ci sono
        #creo la lista per appendere i file con i tweets che hanno le coordinate (perché molte sono null)
        emptyFileWithCoord = []
        # chiamo la classe CreateDF
        cd = CreateDF(entries, df, folder_name, emptyFileWithCoord, coords)
        fileWithCoord = cd.createFilesWithCoord(entries, folder_name, emptyFileWithCoord)
        df = cd.createDf(entries, df, folder_name, fileWithCoord, coords) #creo il df
        
        #saving the dataframe
        new_folder = folder_name[-4:]
        if(os.path.isdir('../csv/'+new_folder)) == False: 
            os.mkdir('../csv/'+new_folder)
        df_path = '../csv/'+new_folder+'/df.csv'
        df.to_csv(df_path)
        print('Created'+ df_path)

In uidFiles_2016 ci sono 2521 files
Created../csv/2016/df.csv


In [5]:
df

,Screen_name,UserID,TweetID,Coords,Lat,Lon,Created_At,Text
x,madikeeper12,868809325,779072240994234368,"[43.72666207, 10.41268069]",43.726662,10.412681,Thu Sep 22 21:37:51 +0000 2016,Cieli infuocati.\n\n#picoftheday #quotesofthed...
x,madikeeper12,868809325,781615843406819329,"[43.72666207, 10.41268069]",43.726662,10.412681,Thu Sep 29 22:05:13 +0000 2016,Prospettive.. \nunite a casa #ilselfone\n#team...
x,madikeeper12,868809325,781870800156499968,"[43.72666207, 10.41268069]",43.726662,10.412681,Fri Sep 30 14:58:19 +0000 2016,"Non occorre essere matti per lavorare qui, ma ..."
x,madikeeper12,868809325,780003801260404736,"[43.7167, 10.3833]",43.716700,10.383300,Sun Sep 25 11:19:32 +0000 2016,RunOnSunDay 🏃🏽‍♀️☀️\n#run #running #runner #ni...
x,madikeeper12,868809325,779443101123260417,"[43.70561, 10.42059]",43.705610,10.420590,Fri Sep 23 22:11:31 +0000 2016,La vita è come la fotografia sono necessari i ...
...,...,...,...,...,...,...,...,...
x,antoniocassisa,358042635,781879291911016448,"[43.7167, 10.3833]",43.716700,10.383300,Fri Sep 30 15:32:04 +0000 2016,"I mì ómini \n#son #figli #boys @ Pisa, Italy h..."
x,SefaMermer,293157588,780753755830677504,"[43.7167, 10.3833]",43.716700,10.383300,Tue Sep 27 12:59:35 +0000 2016,#love #tbt #tagforlikes #TFLers #tweegram #pho...
x,SefaMermer,293157588,780756143668953088,"[43.7167, 10.3833]",43.716700,10.383300,Tue Sep 27 13:09:05 +0000 2016,#love #tbt #tagforlikes #TFLers #tweegram #pho...
x,matteluca89,494389053,779638196258811904,"[43.71544235, 10.40051616]",43.715442,10.400516,Sat Sep 24 11:06:45 +0000 2016,Last saturday I went out with my #chinese teac...
